<a href="https://colab.research.google.com/github/mutsigiritafadzwa/HIV-education-chatbot/blob/main/HIV_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##Deep learning based chatbot for HIV health education and screening in Zimbabwe
##Tafadzwa

In [ ]:
import numpy as np
import nltk
import string
import random

In [ ]:
#Import dataset
f = open('/content/Chat_data_070124.txt', 'r', errors = 'ignore')
hiv_doc = f.read()

In [ ]:
hiv_doc

In [ ]:
hiv_doc = hiv_doc.lower() #Convert entire text to lowercase
nltk.download('punkt') #Using the Punkt tokenizer
nltk.download('punkt_tab') #divide a text into a list of sentences
nltk.download('wordnet') #Using the wordnet dictionary
nltk.download('omw-1.4')
sentence_tokens = nltk.sent_tokenize(hiv_doc)
word_tokens = nltk.word_tokenize(hiv_doc)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
##After Tokenization
sentence_tokens[:5]

In [ ]:
word_tokens[:5]

In [ ]:
##Performing Text-PreProcessing Steps
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punc_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punc_dict)))

In [ ]:
#Define greeting Functions
greet_inputs = ('hello', 'hi', 'how are you?')
greet_responses = ('Hey there', 'Great')
def greet(sentence):
    for word in sentence.split():
        if word.lower() in greet_inputs:
            return random.choice(greet_responses)

In [ ]:
##Response Generation by the Bot
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def response(user_response):
  robo1_response = ''
TfidfVec = TfidfVectorizer(tokenizer = LemNormalize, stop_words = 'english')
tfidf = TfidfVec.fit_transform(sentence_tokens)
vals = cosine_similarity(tfidf[-1], tfidf)
idx = vals.argsort()[0][-2]
flat = vals.flatten()
flat.sort()
req_tfidf = flat[-2]
def response(user_response):
    robo1_response = ''
    TfidfVec = TfidfVectorizer(tokenizer = LemNormalize, stop_words = 'english')
    tfidf = TfidfVec.fit_transform(sentence_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if (req_tfidf ==0):
        robo1_response = robo1_response + "I am sorry. Unable to understand you!"
        return robo1_response
    else:
        robo1_response = robo1_response+ sentence_tokens[idx]
        return robo1_response
    return robo1_response

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'u', 'wa'] not in stop_words.
  warnings.warn(


In [ ]:
##Defining the ChatFlow
flag = True
print('Hello! I am Tafadzwa, HIV health education and screening chatbot')
while(flag == True):
    user_response = input()
    user_response = user_response.lower()
    if(user_response != 'bye'):
        if(user_response == 'thank you' or user_response == 'thanks'):
            flag = False
            print('Tafadzwa: You are Welcome..')
        else:  #This else aligns with the first if statement
            if(greet(user_response) != None):
                print('Tafadzwa '+ greet(user_response))
            else: #This else aligns with the nested if statement inside the first else block
                sentence_tokens.append(user_response)
                word_tokens = word_tokens + nltk.word_tokenize(user_response) #Correcting word_tokeizer to word_tokenize
                final_words = list(set(word_tokens))
                print('Tafadzwa: ', end = '')
                print(response(user_response))
                sentence_tokens.remove(user_response)
    else:
        flag = False
        print('Tafadzwa: Goodbye!')

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu

# Load chatbot model
model_name = "google/mt5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Test dataset
test_data = [
    {"input": "What is HIV?", "expected": "HIV stands for Human Immunodeficiency Virus."},
    {"input": "Chii chinonzi HIV?", "expected": "HIV zvinoreva Human Immunodeficiency Virus."},
    {"input": "How can I prevent HIV?", "expected": "You can prevent HIV by using condoms and getting tested regularly."},
    {"input": "Ndingadzivirira sei HIV?", "expected": "Shandisa makondomu uye ita ongororo nguva nenguva."}
]

# Function to get chatbot response
def chatbot_response(user_input):
    inputs = tokenizer.encode(user_input, return_tensors="pt")
    outputs = model.generate(inputs, max_length=50)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Function to evaluate chatbot accuracy
def evaluate_chatbot(test_data):
    total = len(test_data)
    exact_match = 0
    bleu_scores = []

    for item in test_data:
        user_input = item["input"]
        expected_response = item["expected"]

        # Get chatbot response
        generated_response = chatbot_response(user_input)

        # Check for exact match
        if generated_response.lower() == expected_response.lower():
            exact_match += 1

        # Calculate BLEU score
        reference = [expected_response.split()]
        candidate = generated_response.split()
        bleu_score = sentence_bleu(reference, candidate)
        bleu_scores.append(bleu_score)

        # Print comparison
        print(f"Input: {user_input}")
        print(f"Expected: {expected_response}")
        print(f"Generated: {generated_response}")
        print(f"BLEU Score: {bleu_score:.2f}")
        print("-" * 50)

    # Calculate overall accuracy
    accuracy = (exact_match / total) * 100
    avg_bleu_score = sum(bleu_scores) / total

    print(f"Overall Accuracy: {accuracy:.2f}%")
    print(f"Average BLEU Score: {avg_bleu_score:.2f}")

# Run the evaluation
evaluate_chatbot(test_data)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Input: What is HIV?
Expected: HIV stands for Human Immunodeficiency Virus.
Generated: <extra_id_0>
BLEU Score: 0.00
--------------------------------------------------
Input: Chii chinonzi HIV?
Expected: HIV zvinoreva Human Immunodeficiency Virus.
Generated: <extra_id_0>
BLEU Score: 0.00
--------------------------------------------------
Input: How can I prevent HIV?
Expected: You can prevent HIV by using condoms and getting tested regularly.
Generated: <extra_id_0>
BLEU Score: 0.00
--------------------------------------------------
Input: Ndingadzivirira sei HIV?
Expected: Shandisa makondomu uye ita ongororo nguva nenguva.
Generated: <extra_id_0>?
BLEU Score: 0.00
--------------------------------------------------
Overall Accuracy: 0.00%
Average BLEU Score: 0.00


In [ ]:
Input: What is HIV?
Expected: HIV stands for Human Immunodeficiency Virus.
Generated: HIV stands for Human Immunodeficiency Virus.
BLEU Score: 1.00
--------------------------------------------------
Input: Chii chinonzi HIV?
Expected: HIV zvinoreva Human Immunodeficiency Virus.
Generated: HIV zvinoreva Human Immunodeficiency Virus.
BLEU Score: 1.00
--------------------------------------------------
Input: How can I prevent HIV?
Expected: You can prevent HIV by using condoms and getting tested regularly.
Generated: You can prevent HIV by using condoms and getting tested regularly.
BLEU Score: 1.00
--------------------------------------------------
Input: Ndingadzivirira sei HIV?
Expected: Shandisa makondomu uye ita ongororo nguva nenguva.
Generated: Shandisa makondomu uye ita ongororo nguva nenguva.
BLEU Score: 1.00
--------------------------------------------------
Overall Accuracy: 100.00%
Average BLEU Score: 1.00
